## Accelerating 2:4 sparse models with Huggingface, torch.compile, and semi-structured sparsity. 

This notebook shows how to accelerate an off-the-shelf 2:4 sparse model using pytorch's `to_sparse_semi_structured`

It takes advantage of the model checkpoints offered by neuralmagic: https://huggingface.co/nm-testing/SparseLlama-3-8B-pruned_50.2of4-FP8


Even though we need to pad the matmul shapes from (1, hidden) @ (hidden, output) to (8, hidden) @ (hidden, output) we are still able to achieve speedups on the mlp.up and mlp.gate linear layesr of the FFN

In [30]:
import os
import torch
from torch.sparse import to_sparse_semi_structured

from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer

def timed(fn):
    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)
    start.record()
    result = fn()
    end.record()
    torch.cuda.synchronize()
    return result, start.elapsed_time(end) / 1000


def benchmark(fn, WARMUP=5, N=250):
    time_per_batch = []
    with torch.no_grad():
        # warmup steps
        for _ in range(WARMUP):
            timed(fn)
    
        # benchmark
        for _ in tqdm(range(N)):
            with torch.no_grad():
                _ , time_sec =  timed(fn)
                time_per_batch.append(time_sec)
            
    # each time we generate 128 tokens - 7 for the prompt = 121 tokens at a time.
    total_time = sum(time_per_batch)
    tokens_per_second = 121 * N / total_time
    #print(f"Total time: {total_time:.3f}s | Tokens/second: {tokens_per_second:.3f}")
    print(total_time / N)





In [2]:
torch.sparse.SparseSemiStructuredTensor._FORCE_CUTLASS = False
torch.set_float32_matmul_precision('high')

os.environ["TOKENIZERS_PARALLELISM"] = "false" # silence warnings when compiling

In [3]:
model = AutoModelForCausalLM.from_pretrained("nm-testing/SparseLlama-3-8B-pruned_50.2of4", torch_dtype=torch.float16).cuda()
tokenizer = AutoTokenizer.from_pretrained("nm-testing/SparseLlama-3-8B-pruned_50.2of4")

# Load semi-structured spares
for name, mod in model.named_modules():
    if isinstance(mod, torch.nn.Linear):

        # print out linear layers just FYI
        if '10' in name:
            print(name, mod.weight.shape)

        # these two will show speedups
        if 'mlp.gate' in name or 'mlp.up' in name:    
            mod.weight = torch.nn.Parameter(to_sparse_semi_structured(mod.weight))

# Specify the max length (including both the prompt and the response)
# When calling `generate` with `cache_implementation="static" later, this is also used to create a `StaticCache` object
# with sequence length = `max_length`. The longer the more you will re-use it
model.generation_config.max_length = 128
model.generation_config.pad_token_id = tokenizer.eos_token_id
model.generation_config.cache_implementation = "static"


prompt = "Why dogs are so cute?"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/home/jessecai/local/miniconda3/envs/huggingface-nm-pt/lib/python3.10/site-packages/torch/sparse/semi_structured.py:113: UserWarning: The PyTorch API of SparseSemiStructuredTensor is in prototype stage and will change in the near future. Please open a Github issue for features requests and see our documentation on the torch.sparse module for further information about the project.
  warnings.warn(


model.layers.10.self_attn.q_proj torch.Size([4096, 4096])
model.layers.10.self_attn.k_proj torch.Size([1024, 4096])
model.layers.10.self_attn.v_proj torch.Size([1024, 4096])
model.layers.10.self_attn.o_proj torch.Size([4096, 4096])
model.layers.10.mlp.gate_proj torch.Size([14336, 4096])
model.layers.10.mlp.up_proj torch.Size([14336, 4096])
model.layers.10.mlp.down_proj torch.Size([4096, 14336])


In [ ]:
# without `torch.compile`: each call takes ~ 5.0 seconds (on A100 80G + torch 2.3)
# Total time: 168.715s | Tokens/second: 17.930
outputs = model.generate(**inputs)
response = tokenizer.batch_decode(outputs)[0]
print(response)

In [ ]:
# `torch.compile(model, ...)` is not recommended as you compile callbacks
# and full generate. We recommend compiling only the forward for now. 
# "reduce-overhead" will use cudagraphs.
torch._inductor.config.triton.cudagraph_dynamic_shape_warn_limit = None

model.forward = torch.compile(model.forward, mode="reduce-overhead", fullgraph=True)

In [ ]:
benchmark(model)

In [ ]:
# sanity check we get same output as non-compiled model
outputs = model.generate(**inputs)
response = tokenizer.batch_decode(outputs)[0]
print(response)

## Run torch.compile baseline

In [ ]:
del model
model = AutoModelForCausalLM.from_pretrained("nm-testing/SparseLlama-3-8B-pruned_50.2of4", torch_dtype=torch.float16).cuda()

# set configs again
model.generation_config.max_length = 128
model.generation_config.pad_token_id = tokenizer.eos_token_id
model.generation_config.cache_implementation = "static"

model.forward = torch.compile(model.forward, mode="reduce-overhead", fullgraph=True)
benchmark(model)

In [ ]:
outputs = model.generate(**inputs)
response = tokenizer.batch_decode(outputs)[0]
print(response)

# e2e runs



In [ ]:
!python scripts/hf_eval.py --tasks hellaswag

In [ ]:
!python scripts/hf_eval.py --sparsity semi_sparse_mlp_only --repo_id "nm-testing/SparseLlama-3-8B-pruned_50.2of4" --tasks hellaswag